In [1]:
file = open('input.txt', 'r')
input = file.readlines()
file.close()

In [2]:
#dict with name as key
modules = {}
#queue with tuples (source_module_name, dest_module_name, pulse (True/False))
pulse_queue = []

In [3]:
class FlipFlop:
    def __init__(self, name, outputs) -> None:
        self.name = name
        self.state_on = False
        self.output_modules = outputs
        pass
    
    def set_pulse(self, pulse, input_module = None):
        if pulse:
            return
        else:
            self.state_on = not self.state_on
            #send pulse according to state
            for module in self.output_modules:
                pulse_queue.append((self.name, module, self.state_on))
            return
    
    def to_string(self):
        return f"FlipFlop: {self.name}, output modules: {self.output_modules}, state "+("on" if self.state_on else "off")

In [4]:
class Conjunction:
    def __init__(self, name, outputs) -> None:
        self.name = name
        self.connected_inputs = {}
        self.output_modules = outputs
        pass

    def set_input_module(self, input_module):
        # False -> low, True -> high
        self.connected_inputs[input_module] = False

    def set_pulse(self, pulse, input_module):
        self.connected_inputs[input_module] = pulse
        send = True
        if all(value == True for value in self.connected_inputs.values()):
            send = False
        for module in self.output_modules:
            pulse_queue.append((self.name, module, send))
        return
    
    def to_string(self):
        return f"Conjunction: {self.name}, input modules: {self.connected_inputs}, output modules: {self.output_modules}"

In [5]:
class Broadcast:
    def __init__(self, name, outputs) -> None:
        self.name = name
        self.output_modules = outputs
        pass

    def set_pulse(self, pulse, input_module = None):
        for module in self.output_modules:
            pulse_queue.append((self.name, module, pulse))
        return
    
    def to_string(self):
        return f"Broadcast: {self.name}, output modules: {self.output_modules}"

In [6]:
for line in input:
    #print(line.strip())
    linesplit = line.split()
    outputs = line.strip().split('->')[1].split(',')
    outputs = [s.strip() for s in outputs]
    if linesplit[0] == 'broadcaster':
        modules[linesplit[0]] = Broadcast(linesplit[0], outputs)
    elif linesplit[0][0] == '%':
        modules[linesplit[0][1:]] = FlipFlop(linesplit[0][1:],outputs)
    elif linesplit[0][0] == '&':
        modules[linesplit[0][1:]] = Conjunction(linesplit[0][1:],outputs)

#find input modules for Conjunctions
for module in modules.values():
    if isinstance(module, Conjunction):
        for input in modules.values():
            if module.name in input.output_modules:
                module.set_input_module(input.name)

# Idea
* rx needs to get low pulse from &cs
* &cs sends low pulse, if all inputs send high pulse to rx
    * &kh
    * &lz
    * &tg
    * &hn
* compute lcm of button presses needed for each input

In [7]:
inputs_to_cs = modules['cs'].connected_inputs
first_time_true = {}

In [8]:
#press button i times and work queue
button_presses = 0
for i in range(10000):
    button_presses += 1
    pulse_queue.append(('start','broadcaster',False))
    while (len(pulse_queue) > 0):
        next_pulse = pulse_queue.pop(0)

        #check for input modules of cs:
        if (next_pulse[0] in inputs_to_cs and next_pulse[2]):
            if next_pulse[0] not in first_time_true.keys():
                first_time_true[next_pulse[0]] = button_presses

        if (next_pulse[1] not in modules):
            continue
        else:
            modules[next_pulse[1]].set_pulse(next_pulse[2], next_pulse[0])
        #print(pulse_queue)
    

In [9]:
first_time_true

{'tg': 3769, 'kh': 3889, 'lz': 3917, 'hn': 4013}

In [10]:
from math import lcm

least_common_multiple = 1
for v in first_time_true.values():
    least_common_multiple = lcm(least_common_multiple, v)
print(f"lcm: {least_common_multiple}")

lcm: 230402300925361
